<a href="https://colab.research.google.com/github/YueunKim/Dacon/blob/master/carteen_people_forcast_5~8_(colab)Dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사용 패키지

In [1]:
## https://dacon.io/competitions/official/235743/codeshare/2856?page=1&dtype=recent
## https://dacon.io/competitions/official/235743/codeshare/2817?page=1&dtype=recent

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

## 데이터 로드

In [4]:
train = pd.read_csv('/content/gdrive/MyDrive/dacon/carteen_people/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/gdrive/MyDrive/dacon/carteen_people/test.csv', encoding = 'utf-8')
submission = pd.read_csv('/content/gdrive/MyDrive/dacon/carteen_people/sample_submission.csv', encoding = 'utf-8')

## 데이터

In [5]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [6]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [7]:
print('train shape:', train.shape, '\n', 'test shape:', test.shape)

train shape: (1205, 12) 
 test shape: (50, 10)


In [8]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [9]:
submission.columns = ['date', 'target_ln', 'target_dn']

## 조식메뉴 drop

In [10]:
train = train.drop('조식메뉴', axis=1)
test = test.drop('조식메뉴', axis=1)

## 컬럼명 수정 

In [11]:
train.columns = ['date', 'dow', 'employees', 'dayoff', 'bustrip', 'ovtime', 'remote', 'ln', 'dn', 'target_ln', 'target_dn']
test.columns = ['date', 'dow', 'employees', 'dayoff', 'bustrip', 'ovtime', 'remote', 'ln', 'dn']

## 날짜, 요일 수정

In [12]:
def to_datetime(df, date):
    df['date'] = pd.to_datetime(df[date])
    df['dow'] = pd.to_datetime(df[date]).dt.weekday + 1 # dt.weekday는 월요일 = 0 
    
to_datetime(train, 'date')
to_datetime(test, 'date')

In [13]:
train['mon'] = pd.DatetimeIndex(train['date']).month
test['mon'] = pd.DatetimeIndex(test['date']).month

train['day'] = pd.DatetimeIndex(train['date']).day
test['day'] = pd.DatetimeIndex(test['date']).day

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1205 non-null   datetime64[ns]
 1   dow        1205 non-null   int64         
 2   employees  1205 non-null   int64         
 3   dayoff     1205 non-null   int64         
 4   bustrip    1205 non-null   int64         
 5   ovtime     1205 non-null   int64         
 6   remote     1205 non-null   float64       
 7   ln         1205 non-null   object        
 8   dn         1205 non-null   object        
 9   target_ln  1205 non-null   float64       
 10  target_dn  1205 non-null   float64       
 11  mon        1205 non-null   int64         
 12  day        1205 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(7), object(2)
memory usage: 122.5+ KB


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       50 non-null     datetime64[ns]
 1   dow        50 non-null     int64         
 2   employees  50 non-null     int64         
 3   dayoff     50 non-null     int64         
 4   bustrip    50 non-null     int64         
 5   ovtime     50 non-null     int64         
 6   remote     50 non-null     float64       
 7   ln         50 non-null     object        
 8   dn         50 non-null     object        
 9   mon        50 non-null     int64         
 10  day        50 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(7), object(2)
memory usage: 4.4+ KB


#### 휴가자, 재택근무자는 중식/석식에 참여하지 않을 것이므로 본사정원수에서 차감한다.

#### 즉, 본사정원수 - 휴가자 - 재택근무자

#### 식사참여율 = 식사계 / 식사가능자수

In [16]:
train['caneat'] = train['employees'] - train['dayoff'] - train['remote']
test['caneat'] = test['employees'] - test['dayoff'] - test['remote']

train['rate_ln'] = train['target_ln'] / train['caneat']
train['rate_dn'] = train['target_dn'] / train['caneat']

## 피쳐 선택

In [17]:
features = ['mon', 'day', 'dow', 'caneat', 'bustrip', 'ovtime']
labels = ['target_ln',	'target_dn', 'rate_ln', 'rate_dn']

train = train[features+labels]
test = test[features]

In [18]:
train.head()

,mon,day,dow,caneat,bustrip,ovtime,target_ln,target_dn,rate_ln,rate_dn
0,2,1,1,2551.0,150,238,1039.0,331.0,0.407291,0.129753
1,2,2,2,2551.0,173,319,867.0,560.0,0.339867,0.219522
2,2,3,3,2545.0,180,111,1017.0,573.0,0.399607,0.225147
3,2,4,4,2497.0,220,355,978.0,525.0,0.391670,0.210252
4,2,5,5,2323.0,181,34,925.0,330.0,0.398192,0.142058


In [19]:
test.head()

,mon,day,dow,caneat,bustrip,ovtime
0,1,27,3,2537.0,182,5
1,1,28,4,2531.0,212,409
2,1,29,5,2419.0,249,0
3,2,1,1,2494.0,154,538
4,2,2,2,2548.0,186,455


In [20]:
x = train[['mon', 'day', 'dow', 'caneat', 'bustrip', 'ovtime']]
y_ln = train['target_ln']
y_dn = train['target_dn']

test_x = test[['mon', 'day', 'dow', 'caneat', 'bustrip', 'ovtime']]

## CatBoost 

In [21]:
!pip3 install catboost

from catboost import CatBoostRegressor

     |████████████████████████████████| 69.2 MB 4.6 kB/s 


In [23]:
# cat_lunch = CatBoostRegressor()
# cat_dinner = CatBoostRegressor()

# cat_lunch = CatBoostRegressor( depth=10, n_estimators = 200 )
# cat_dinner = CatBoostRegressor( depth=10, n_estimators = 200 )

# cat_lunch = CatBoostRegressor( depth=10, n_estimators = 20 )
# cat_dinner = CatBoostRegressor( depth=10, n_estimators = 20 )

cat_lunch = CatBoostRegressor(
         loss_function='MAE',
         n_estimators=1000, 
         learning_rate=0.05, 
         random_state=42)
cat_dinner = CatBoostRegressor(
         loss_function='MAE',
         n_estimators=1000, 
         learning_rate=0.05, 
         random_state=42)

cat_lunch.fit(x, y_ln,verbose=False, plot=True)
cat_dinner.fit(x, y_dn,verbose=False, plot=True)

pred_lunch = cat_lunch.predict(test_x)
pred_dinner = cat_dinner.predict(test_x)

submission['target_ln'] = pred_lunch
submission['target_dn'] = pred_dinner

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## 결과 제출

In [24]:
submission.columns = ['일자', '중식계', '석식계']

In [25]:
submission.to_csv('sub_8.csv', index=False, encoding = 'utf-8')

In [26]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [28]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/sub_8.csv', 
'', 
'235743', 
'yuniyuni',
'CatBoost 사용, 파라미터 조정 ' )

{'isSubmitted': True, 'detail': 'Success'}
